In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

In [ ]:
# Search for all image files in testing set...
all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/Lab/*/*/*/Images/Quad*/*.jpg'))
len(all_images)

In [ ]:
# meta-parameters
'''
@param blockSize Size of a pixel neighborhood 
       that is used to calculate a threshold value for the pixel.
@param C Constant subtracted from the mean or weighted mean 
       (see the details below). Normally, it is positive but may be zero 
       or negative as well.
@param k_size morphology structuring element size
'''
blockSize  = 301
C_constant = 2
k_size     = 11

min_area = 40000
max_area = 300000

bottom_cut = 150
horiz_cut = 200

In [ ]:
for i in range(len(all_images)):
    im   = cv2.imread(all_images[i])
    rgb  = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    th1  = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize, C_constant)

    #kernel = np.ones((k_size, k_size), np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_size, k_size))
    erosion = cv2.erode(th1, kernel, iterations=2)
    close = cv2.dilate(erosion, kernel, iterations=1)

    '''
    @param mode cv2.RETR_EXTERNAL retrieves only the extreme outer contours.
    @param method cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, 
           and diagonal segments and leaves only their end points. For example, 
           an up-right rectangular contour is encoded with 4 points.
    '''
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        area = cv2.contourArea(c)
        if area > min_area: #and area < max_area:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(rgb, (x, y), (x + w, y + h), (36, 255, 12), 5)
            close[y:y+h, x:x+w] = 0

    fig, axes = plt.subplots(1, 2, figsize=(16, 12))
    axes[0].imshow(rgb)
    
    close[:, :horiz_cut] = 0
    close[:, close.shape[1] - horiz_cut:] = 0
    close[close.shape[0] - bottom_cut:, :] = 0
    
    axes[1].imshow(close)
    for i in range(len(axes.flat)):
        axes.flat[i].axis('off')
    plt.show() #pause(0.1)

In [ ]:
#horiz_cut = 200
close[:, :horiz_cut] = 0
close[:, close.shape[1] - horiz_cut:] = 0
close[close.shape[0] - bottom_cut:, :] = 0
plt.imshow(close)

In [ ]:
close.shape

In [ ]:
rgb.shape